# End-to-End

## 1. Problem (mini project)

Mini Project: FIFA - MoneyBall

Perform an end-to-end analysis putting into practice what you have learned so far. You will apply statistical or machine learning techniques and present your results to the class.



In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings('ignore')

## 2. Getting Data

In [2]:
data = pd.read_csv('fifa21_male2.csv')
data.head()

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


In [3]:
data.shape

(17125, 107)

#### Abbreviations
OVA = overall scores
<br>POT = potential scores
<br>W/F = weak foot
<br>SM = skill moves
<br>IR = international reputation
<br>GK = goalkeeper
<br>CF = center forward

In [4]:
data.columns

Index(['ID', 'Name', 'Age', 'OVA', 'Nationality', 'Club', 'BOV', 'BP',
       'Position', 'Player Photo',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'Gender'],
      dtype='object', length=107)

## 3. Cleaning/Wrangling/EDA

In [5]:
# Standarize name of columns and remove ID column
data.columns = [col_name.lower().replace(' ', '_') for col_name in data.columns]
data = data.drop(data.columns[0], axis=1)
data.head()

,name,age,ova,nationality,club,bov,bp,position,player_photo,club_logo,flag_photo,pot,team_&_contract,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,gender
0,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


In [6]:
for col_name in data.columns: 
    print(col_name)

name
age
ova
nationality
club
bov
bp
position
player_photo
club_logo
flag_photo
pot
team_&_contract
height
weight
foot
growth
joined
loan_date_end
value
wage
release_clause
contract
attacking
crossing
finishing
heading_accuracy
short_passing
volleys
skill
dribbling
curve
fk_accuracy
long_passing
ball_control
movement
acceleration
sprint_speed
agility
reactions
balance
power
shot_power
jumping
stamina
strength
long_shots
mentality
aggression
interceptions
positioning
vision
penalties
composure
defending
marking
standing_tackle
sliding_tackle
goalkeeping
gk_diving
gk_handling
gk_kicking
gk_positioning
gk_reflexes
total_stats
base_stats
w/f
sm
a/w
d/w
ir
pac
sho
pas
dri
def
phy
hits
ls
st
rs
lw
lf
cf
rf
rw
lam
cam
ram
lm
lcm
cm
rcm
rm
lwb
ldm
cdm
rdm
rwb
lb
lcb
cb
rcb
rb
gk
gender


A lot of meanings of the columns can be found in https://sofifa.com/

In [7]:
# columns to drop

drop_columns = ["position","player_photo","club_logo","flag_photo","team_&_contract","joined",
                "contract","ls","st","rs","lw","lf","cf","rf","rw","lam","cam","ram","lm","lcm",
                "cm","rcm","rm","lwb","ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb","gk","gender"]

# I think in a further step we should also drop the "name" column for the linear regression


In [8]:
data = data.drop(drop_columns, axis=1)

In [9]:
# rename rest of the columns with their meaning

data = data.rename(columns={"ova":"overall_scores",
                        "bov":"best_overall",
                        "pot":"potential_scores",
                        "w/f":"weak_foot",
                        "sm":"skill_moves",
                        "ir":"international_reputation",
                        "gk":"goalkeeper",
                        "cf":"center_forward",
                        "lde":"loan_date_end",
                        "pac":"pace",
                        "bp":"better_position", 
                        "sm":"skilled_moves",
                        "a/w":"attacking_work_rate",
                        "d/w":"defensive_work_rate",
                        "sho":"shooting",
                        "pas":"passing",
                        "dri":"dribbling",
                        "def":"defending",
                        "phy":"physical"})
data.head()

,name,age,overall_scores,nationality,club,best_overall,better_position,potential_scores,height,weight,foot,growth,loan_date_end,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,weak_foot,skilled_moves,attacking_work_rate,defensive_work_rate,international_reputation,pace,shooting,passing,dribbling,defending,physical,hits
0,G. Pasquale,33,69,Italy,Udinese,71,LWB,69,"6'0""",181lbs,Left,0,NaN,€625K,€7K,€0,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4
1,Luis García,37,71,Spain,KAS Eupen,70,CM,71,"5'10""",143lbs,Right,0,NaN,€600K,€7K,€1.1M,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4
2,J. Cole,33,71,England,Coventry City,71,CAM,71,"5'9""",161lbs,Right,0,NaN,€1.1M,€15K,€0,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11
3,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,82,"5'11""",165lbs,Right,14,NaN,€0,€0,€0,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3
4,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,81,"5'7""",150lbs,Right,0,NaN,€5.5M,€12K,€7.2M,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149


In [10]:
# NAN VALUES

data.isnull().sum()

name               0
age                0
overall_scores     0
nationality        0
club              23
                  ..
passing            0
dribbling          0
defending          0
physical           0
hits               0
Length: 71, dtype: int64

In [11]:
data.isnull().sum().sum()

17259

In [12]:
# show all the columns with their nan count
for col_name in data.columns: 
    print(col_name, data[col_name].isnull().sum())

name 0
age 0
overall_scores 0
nationality 0
club 23
best_overall 0
better_position 0
potential_scores 0
height 0
weight 0
foot 0
growth 0
loan_date_end 16215
value 0
wage 0
release_clause 0
attacking 0
crossing 0
finishing 0
heading_accuracy 0
short_passing 0
volleys 58
skill 0
dribbling dribbling    0
dribbling    0
dtype: int64
curve 58
fk_accuracy 0
long_passing 0
ball_control 0
movement 0
acceleration 0
sprint_speed 0
agility 58
reactions 0
balance 58
power 0
shot_power 0
jumping 58
stamina 0
strength 0
long_shots 0
mentality 0
aggression 0
interceptions 7
positioning 7
vision 58
penalties 0
composure 423
defending defending    0
defending    0
dtype: int64
marking 0
standing_tackle 0
sliding_tackle 58
goalkeeping 0
gk_diving 0
gk_handling 0
gk_kicking 0
gk_positioning 0
gk_reflexes 0
total_stats 0
base_stats 0
weak_foot 0
skilled_moves 0
attacking_work_rate 89
defensive_work_rate 89
international_reputation 0
pace 0
shooting 0
passing 0
dribbling dribbling    0
dribbling    0
dtyp

In [13]:
# loan_date_end has a very big count of nans: 16215, I would drop this column, and also composure (423)

In [14]:
data = data.drop(["loan_date_end","composure"], axis = 1)

In [15]:
data.isnull().sum().sum()

621

In [16]:
# we still have a lot of nans, I would keep the "club" column because it's important but the rest of the columns with nans I would drop
data["club"] = data["club"].fillna("unknown")
data.shape

(17125, 69)

In [17]:
data = data.dropna(axis =1)
data.shape

(17125, 58)

In [18]:
data.isnull().sum().sum()

0

In [19]:
# We still have 58 columns to work with. I feel like it's too much... especially after what Xisca said yesterday
# so if you want to drop more, fine for me ;)

In [20]:
# let's check the type of the columns
data.dtypes

name                        object
age                          int64
overall_scores               int64
nationality                 object
club                        object
best_overall                 int64
better_position             object
potential_scores             int64
height                      object
weight                      object
foot                        object
growth                       int64
value                       object
wage                        object
release_clause              object
attacking                    int64
crossing                     int64
finishing                    int64
heading_accuracy             int64
short_passing                int64
skill                        int64
dribbling                    int64
fk_accuracy                  int64
long_passing                 int64
ball_control                 int64
movement                     int64
acceleration                 int64
sprint_speed                 int64
reactions           

In [21]:
# Columns that are objects and should be numerical:
# height, weight, value, wage, release_clause, weak_foot, skilled_moves, international_reputation   

In [22]:
# height
data["height"].value_counts()

6'0"     2583
5'10"    2418
5'9"     2027
5'11"    1944
6'2"     1878
6'1"     1805
5'8"      960
6'3"      953
5'7"      893
6'4"      719
5'6"      307
6'5"      254
5'5"      185
6'6"      111
5'4"       28
6'7"       27
6'8"       12
5'3"       12
5'2"        5
6'9"        2
5'1"        2
Name: height, dtype: int64

In [23]:
# I'm going to create a new column height_cm and convert the values to cm. For that, first I remove the " symbol

data["height"] = data["height"].str.replace('"',"")
data["height"].value_counts()

6'0     2583
5'10    2418
5'9     2027
5'11    1944
6'2     1878
6'1     1805
5'8      960
6'3      953
5'7      893
6'4      719
5'6      307
6'5      254
5'5      185
6'6      111
5'4       28
6'7       27
6'8       12
5'3       12
5'2        5
6'9        2
5'1        2
Name: height, dtype: int64

In [24]:
data["height_cm"] = data["height"].apply(lambda x: int(x.split("'")[0])*30.48 + int(x.split("'")[1])*2.54)
data.head()

,name,age,overall_scores,nationality,club,best_overall,better_position,potential_scores,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,skill,dribbling,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,reactions,power,shot_power,stamina,strength,long_shots,mentality,aggression,penalties,defending,marking,standing_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,weak_foot,skilled_moves,international_reputation,pace,shooting,passing,dribbling,defending,physical,hits,height_cm
0,G. Pasquale,33,69,Italy,Udinese,71,LWB,69,6'0,181lbs,Left,0,€625K,€7K,€0,313,75,50,59,71,338,73,60,69,71,347,68,74,69,347,74,69,68,68,320,72,50,208,70,69,56,14,5,15,10,12,1929,408,3 ★,2★,2 ★,71,59,70,71,68,69,4,182.88
1,Luis García,37,71,Spain,KAS Eupen,70,CM,71,5'10,143lbs,Right,0,€600K,€7K,€1.1M,337,68,64,61,76,369,69,79,71,71,305,56,50,65,324,75,64,60,71,362,71,75,153,70,43,56,9,12,13,11,11,1906,385,4 ★,3★,1 ★,53,69,73,69,58,63,4,177.80
2,J. Cole,33,71,England,Coventry City,71,CAM,71,5'9,161lbs,Right,0,€1.1M,€15K,€0,337,80,64,41,77,387,79,77,69,78,295,48,42,59,284,72,29,56,69,317,69,66,99,35,34,51,9,6,13,16,7,1770,354,4 ★,4★,2 ★,45,68,76,77,36,52,11,175.26
3,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,82,5'11,165lbs,Right,14,€0,€0,€0,264,54,70,60,80,255,68,46,64,77,176,59,62,55,239,63,51,66,59,271,59,70,75,34,41,68,5,21,64,21,21,1348,369,3 ★,1★,1 ★,61,66,66,69,47,60,3,180.34
4,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,81,5'7,150lbs,Right,0,€5.5M,€12K,€7.2M,367,75,69,54,90,408,85,70,83,90,346,61,56,75,297,67,58,62,70,370,58,71,181,68,57,45,6,13,6,13,7,2014,420,4 ★,4★,4 ★,58,70,85,85,63,59,149,170.18


In [25]:
# now I drop the "height" column
data = data.drop("height", axis = 1)
data.head()

,name,age,overall_scores,nationality,club,best_overall,better_position,potential_scores,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,skill,dribbling,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,reactions,power,shot_power,stamina,strength,long_shots,mentality,aggression,penalties,defending,marking,standing_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,weak_foot,skilled_moves,international_reputation,pace,shooting,passing,dribbling,defending,physical,hits,height_cm
0,G. Pasquale,33,69,Italy,Udinese,71,LWB,69,181lbs,Left,0,€625K,€7K,€0,313,75,50,59,71,338,73,60,69,71,347,68,74,69,347,74,69,68,68,320,72,50,208,70,69,56,14,5,15,10,12,1929,408,3 ★,2★,2 ★,71,59,70,71,68,69,4,182.88
1,Luis García,37,71,Spain,KAS Eupen,70,CM,71,143lbs,Right,0,€600K,€7K,€1.1M,337,68,64,61,76,369,69,79,71,71,305,56,50,65,324,75,64,60,71,362,71,75,153,70,43,56,9,12,13,11,11,1906,385,4 ★,3★,1 ★,53,69,73,69,58,63,4,177.80
2,J. Cole,33,71,England,Coventry City,71,CAM,71,161lbs,Right,0,€1.1M,€15K,€0,337,80,64,41,77,387,79,77,69,78,295,48,42,59,284,72,29,56,69,317,69,66,99,35,34,51,9,6,13,16,7,1770,354,4 ★,4★,2 ★,45,68,76,77,36,52,11,175.26
3,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,82,165lbs,Right,14,€0,€0,€0,264,54,70,60,80,255,68,46,64,77,176,59,62,55,239,63,51,66,59,271,59,70,75,34,41,68,5,21,64,21,21,1348,369,3 ★,1★,1 ★,61,66,66,69,47,60,3,180.34
4,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,81,150lbs,Right,0,€5.5M,€12K,€7.2M,367,75,69,54,90,408,85,70,83,90,346,61,56,75,297,67,58,62,70,370,58,71,181,68,57,45,6,13,6,13,7,2014,420,4 ★,4★,4 ★,58,70,85,85,63,59,149,170.18


In [26]:
# weight: the same, convert to kg

In [27]:
data["weight"] = data["weight"].str.replace("lbs","")
data["weight"].value_counts()

154    1342
165    1297
176    1028
159     925
172     898
161     858
163     852
168     818
170     759
150     683
174     615
157     605
181     567
152     500
143     478
179     468
183     454
148     450
185     403
187     377
146     367
141     298
190     274
194     246
139     202
192     192
198     168
137     153
132     146
196     121
134     108
201      79
203      76
205      46
209      42
128      40
207      37
130      32
212      18
121      14
126      13
123      12
214      12
216      12
220      10
227       5
218       4
119       4
225       4
223       3
243       2
229       2
117       2
110       1
234       1
236       1
115       1
Name: weight, dtype: int64

In [28]:
data["weight_kg"] = data["weight"].apply(lambda x: round((int(x)*0.45359237), 2))
data = data.drop("weight", axis = 1)
data.head()

,name,age,overall_scores,nationality,club,best_overall,better_position,potential_scores,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,skill,dribbling,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,reactions,power,shot_power,stamina,strength,long_shots,mentality,aggression,penalties,defending,marking,standing_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,weak_foot,skilled_moves,international_reputation,pace,shooting,passing,dribbling,defending,physical,hits,height_cm,weight_kg
0,G. Pasquale,33,69,Italy,Udinese,71,LWB,69,Left,0,€625K,€7K,€0,313,75,50,59,71,338,73,60,69,71,347,68,74,69,347,74,69,68,68,320,72,50,208,70,69,56,14,5,15,10,12,1929,408,3 ★,2★,2 ★,71,59,70,71,68,69,4,182.88,82.10
1,Luis García,37,71,Spain,KAS Eupen,70,CM,71,Right,0,€600K,€7K,€1.1M,337,68,64,61,76,369,69,79,71,71,305,56,50,65,324,75,64,60,71,362,71,75,153,70,43,56,9,12,13,11,11,1906,385,4 ★,3★,1 ★,53,69,73,69,58,63,4,177.80,64.86
2,J. Cole,33,71,England,Coventry City,71,CAM,71,Right,0,€1.1M,€15K,€0,337,80,64,41,77,387,79,77,69,78,295,48,42,59,284,72,29,56,69,317,69,66,99,35,34,51,9,6,13,16,7,1770,354,4 ★,4★,2 ★,45,68,76,77,36,52,11,175.26,73.03
3,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,82,Right,14,€0,€0,€0,264,54,70,60,80,255,68,46,64,77,176,59,62,55,239,63,51,66,59,271,59,70,75,34,41,68,5,21,64,21,21,1348,369,3 ★,1★,1 ★,61,66,66,69,47,60,3,180.34,74.84
4,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,81,Right,0,€5.5M,€12K,€7.2M,367,75,69,54,90,408,85,70,83,90,346,61,56,75,297,67,58,62,70,370,58,71,181,68,57,45,6,13,6,13,7,2014,420,4 ★,4★,4 ★,58,70,85,85,63,59,149,170.18,68.04


In [29]:
# value
data["value"].value_counts()

€1.1M     500
€0        457
€1M       347
€325K     342
€450K     341
         ... 
€52M        1
€72.5M      1
€67.5M      1
€46M        1
€5K         1
Name: value, Length: 216, dtype: int64

In [30]:
def clean_value(x):
    x = x.replace("€","")
    x = x.replace(".","")
    x = x.replace("K","000")
    x = x.replace("M","000000")
    x = int(x)
    return x

data["value"] = list(map(clean_value, data["value"])) 
data["value"].value_counts()


11000000     559
0            457
12000000     393
1000000      347
325000       342
            ... 
595000000      1
465000000      1
52000000       1
725000000      1
5000           1
Name: value, Length: 180, dtype: int64

In [31]:
# wage
data["wage"].value_counts()

€2K      2453
€500     1746
€3K      1710
€1K      1293
€4K      1091
         ... 
€200K       2
€370K       1
€560K       1
€290K       1
€250        1
Name: wage, Length: 142, dtype: int64

In [32]:
data["wage"] = list(map(clean_value, data["wage"])) # apply the same function
data["wage"].value_counts()

2000      2453
500       1746
3000      1710
1000      1293
4000      1091
          ... 
200000       2
370000       1
560000       1
290000       1
250          1
Name: wage, Length: 142, dtype: int64

In [33]:
# release_clause
data["release_clause"].value_counts()

€0        1785
€1.1M      525
€1.2M      441
€1.3M      421
€1.4M      389
          ... 
€87M         1
€49.5M       1
€70.9M       1
€49.8M       1
€367K        1
Name: release_clause, Length: 1200, dtype: int64

In [34]:
data["release_clause"] = list(map(clean_value, data["release_clause"])) # again apply the same function
data["release_clause"].value_counts()

0            1785
11000000      551
12000000      465
13000000      433
14000000      399
             ... 
644000000       1
738000          1
503000000       1
405000000       1
367000          1
Name: release_clause, Length: 1155, dtype: int64

In [ ]:
#  weak_foot, skilled_moves, international_reputation : still need to be changed to numeric

## 4. Processing Data

## 5. Modeling

### Linear Regression

## 6. Model Validation

## 7. Reporting